<a href="https://colab.research.google.com/github/SIVASKANDAN-P/PYTHON-CODE-HINT/blob/main/Code_solution_experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install datasets
!pip install -U datasets
!pip install bert_score
!pip install peft
!pip install accelerate

In [45]:
#importing necessary libraries
from datasets import load_dataset, load_dataset_builder
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from google.colab import drive
drive.mount('/content/drive')
from bert_score import score
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
#    prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from transformers import T5Tokenizer, T5ForConditionalGeneration


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
#Load dataset
dataset = load_dataset("openbmb/UltraInteract_sft",split="train")
#convert to pandas dataframe
data = pd.DataFrame(dataset)

In [47]:
#load dataset using dataset builder
#load_dataset_builder("openbmb/UltraInteract_sft")
#printing the featured in dataset
#ds_builder.info.features

In [48]:
#number of fields in dataframe
data.columns

Index(['task', 'dataset', 'instruction', 'response', 'id', 'parent_id'], dtype='object')

In [49]:
#The number of examples we have for each task
data.task.value_counts()

,count
task,
Coding,114826
Math_PoT,83913
Math_CoT,78349
Logic,11491


Adding colums for getting the length of instruction and response column for getting idea on required context length in LLM. And selecting examples only satisfying context length

In [50]:
def count_words(text):
    words = text.split()  # Split the string by whitespace
    return len(words)

In [51]:
#Adding colums for getting the length of instruction column
data['ins-len'] = data['instruction'].apply(lambda x: len(x.split()))
#Adding colums for getting the length of Response column
data['res-len'] = data['response'].apply(lambda x: len(x.split()))
#Adding colums for sum of the length of instruction and response columns
data['max-len'] = data['res-len'] + data['ins-len']

In [52]:
#printing number of records with max length less than 300
data3= data[data['max-len'] < 300]
print(data[data['max-len'] < 300].task.value_counts())

task
Math_PoT    62197
Math_CoT    61845
Coding      21110
Logic        2273
Name: count, dtype: int64


In [53]:
#max len in the full dataset
data['max-len'].max()

1915

In [54]:
#max length in filtred dataset
data3['max-len'].max()

299

In [55]:
print(data3.task.value_counts())

task
Math_PoT    62197
Math_CoT    61845
Coding      21110
Logic        2273
Name: count, dtype: int64


In [56]:
#Loading the tokenizer for model
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [57]:
#save dataset to drive
#data.to_csv(r'MyDrive/ULTRAINTERACT/ultrainteract.csv')

In [58]:
train_dataset = dataset.train_test_split(test_size=0.1)["train"]
eval_dataset = dataset.train_test_split(test_size=0.1)["test"]

In [76]:
model1 = "google/flan-t5-large"
model2 = "google/flan-t5-small"
model3 = "google/flan-t5-base"
model4 = "google/flan-t5-xl"
model5 = "google/flan-t5-xxl"
model6 = "codellama/CodeLlama-7b-hf"

In [83]:
base_model = model5

loading t5 models using cpu

In [79]:

tokenizer = T5Tokenizer.from_pretrained(base_model)
model = T5ForConditionalGeneration.from_pretrained(base_model)

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie alt sind Sie?</s>


loading t5 models using GPU

In [ ]:
# pip install accelerate

tokenizer = T5Tokenizer.from_pretrained(base_model)
model = T5ForConditionalGeneration.from_pretrained(base_model, device_map="auto")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

In [61]:
model.num_parameters()/ 1000000000

0.78315008

In [85]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    #load_in_8bit=True,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model)

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [64]:
model_size = {}

In [86]:
#model size in billion
model_size[base_model] = model.num_parameters()/ 1000000000

In [87]:
model_size

{'google/flan-t5-large': 0.78315008,
 'google/flan-t5-small': 0.076961152,
 'google/flan-t5-base': 0.247577856,
 'google/flan-t5-xl': 2.849757184,
 'codellama/CodeLlama-7b-hf': 6.738546688}

In [88]:
model_size_df = pd.DataFrame(list(model_size.items()),columns = ['Model','Size'])

In [101]:
model_size_df.to_csv(r'model_size_df.csv')

In [99]:
# prompt: list the directory current

!cd drive
!cd MyDrive
!cd ULTRAINTERACT


/bin/bash: line 1: cd: MyDrive: No such file or directory
/bin/bash: line 1: cd: ULTRAINTERACT: No such file or directory


In [98]:
!ls


drive  sample_data
